In [1]:
#monter Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers datasets sentencepiece

In [ ]:
!pip uninstall -y sentence-transformers


Found existing installation: sentence-transformers 4.1.0
Uninstalling sentence-transformers-4.1.0:
  Successfully uninstalled sentence-transformers-4.1.0


In [ ]:
!pip install --upgrade --quiet transformers==4.40.1 datasets sentencepiece sympy==1.13.1


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 57.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 4.1.0 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.40.1 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", bu

In [ ]:
!pip install --upgrade transformers==4.41.2 sympy==1.13.1 --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 65.8 MB/s eta 0:00:00


In [ ]:
# Forcer les versions compatibles
!pip install transformers==4.40.1 datasets sentencepiece sympy==1.13.1 --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 68.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 90.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 72.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 4.1.0 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.40.1 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 re

In [2]:
#import
import pandas as pd
from datasets import Dataset
from transformers import (
    MarianTokenizer,
    MarianMTModel,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq
)
import torch

In [3]:
# Charger et prétraiter le dataset texte

# Charger le CSV
df = pd.read_csv('/content/drive/MyDrive/Dataset-bara/dataset_bara.csv')

# Exemple de prévisualisation
df.head()

,id,texte_mg_officiel,texte_bara,Unnamed: 3,audio_bara
0,1.0,Salama e,Nareo koa?,./Dataset_bara/malgache_officiel_mp3/1.mp3,NaN
1,2.0,Ilo no anarako,Ilo agnarako,./Dataset_bara/malgache_officiel_mp3/2.mp3,NaN
2,3.0,20 taona aho,20 tao iaho,./Dataset_bara/malgache_officiel_mp3/3.mp3,NaN
3,4.0,mianatra eny amin'ny oniversite Andrainjato aho,mianatsy egny aminy oniversite andrainjato iaho,./Dataset_bara/malgache_officiel_mp3/4.mp3,NaN
4,5.0,manana zandry iray dia zoky iray,mana zay raiky vo zoky raiky,./Dataset_bara/malgache_officiel_mp3/5.mp3,NaN


In [ ]:
# 1. Charger et dupliquer le dataset

In [4]:
# Renommer pour cohérence
df1 = df[['texte_mg_officiel', 'texte_bara']].rename(columns={
    'texte_mg_officiel': 'source',
    'texte_bara': 'target'
})
df1['lang_tag'] = '>>bara<<'  # Traduction vers dialecte bara
df1.head()

,source,target,lang_tag
0,Salama e,Nareo koa?,>>bara<<
1,Ilo no anarako,Ilo agnarako,>>bara<<
2,20 taona aho,20 tao iaho,>>bara<<
3,mianatra eny amin'ny oniversite Andrainjato aho,mianatsy egny aminy oniversite andrainjato iaho,>>bara<<
4,manana zandry iray dia zoky iray,mana zay raiky vo zoky raiky,>>bara<<


In [5]:
# Créer la version inversée
df2 = df[['texte_mg_officiel', 'texte_bara']].rename(columns={
    'texte_bara': 'source',
    'texte_mg_officiel': 'target'
})
df2['lang_tag'] = '>>officiel<<'  # Traduction vers malgache officiel
df2.head()

,target,source,lang_tag
0,Salama e,Nareo koa?,>>officiel<<
1,Ilo no anarako,Ilo agnarako,>>officiel<<
2,20 taona aho,20 tao iaho,>>officiel<<
3,mianatra eny amin'ny oniversite Andrainjato aho,mianatsy egny aminy oniversite andrainjato iaho,>>officiel<<
4,manana zandry iray dia zoky iray,mana zay raiky vo zoky raiky,>>officiel<<


In [6]:
# Fusionner les deux
df_final = pd.concat([df1, df2], ignore_index=True)
df_final.head()

,source,target,lang_tag
0,Salama e,Nareo koa?,>>bara<<
1,Ilo no anarako,Ilo agnarako,>>bara<<
2,20 taona aho,20 tao iaho,>>bara<<
3,mianatra eny amin'ny oniversite Andrainjato aho,mianatsy egny aminy oniversite andrainjato iaho,>>bara<<
4,manana zandry iray dia zoky iray,mana zay raiky vo zoky raiky,>>bara<<


In [7]:
# Ajouter les tags au début du texte source
df_final['source'] = df_final['lang_tag'] + " " + df_final['source']
df_final = df_final[['source', 'target']]  # On n'a plus besoin du tag séparé
df_final.head()

,source,target
0,>>bara<< Salama e,Nareo koa?
1,>>bara<< Ilo no anarako,Ilo agnarako
2,>>bara<< 20 taona aho,20 tao iaho
3,>>bara<< mianatra eny amin'ny oniversite Andra...,mianatsy egny aminy oniversite andrainjato iaho
4,>>bara<< manana zandry iray dia zoky iray,mana zay raiky vo zoky raiky


In [8]:
df_final = df_final.dropna(subset=['source', 'target'])

In [ ]:
# 2. Tokenisation

In [9]:
model_name = "Helsinki-NLP/opus-mt-en-mg"  # base multilingue

tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/796k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/806k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:177: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/291M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/291M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [10]:
# Tokenisation
def tokenize_function(examples):
    model_inputs = tokenizer(
        examples["source"], max_length=128, truncation=True, padding="max_length"
    )
    labels = tokenizer(
        examples["target"], max_length=128, truncation=True, padding="max_length"
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [11]:
# ---------------------
# 3. Convertir en dataset Hugging Face
# ---------------------
hf_dataset = Dataset.from_pandas(df_final)
tokenized_dataset = hf_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/584 [00:00<?, ? examples/s]

In [12]:
# 4. Entraînement
# ---------------------
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="/content/marian-bara",
    save_steps=500,
    eval_steps=500,
    logging_steps=100,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=torch.cuda.is_available(),
    learning_rate=5e-5,
    report_to="none"  # 👈 Désactive Wandb, TensorBoard, etc.
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset.select(range(100)),  # petite validation
    tokenizer=tokenizer,
    data_collator=data_collator,
)
trainer.train()

/tmp/ipython-input-12-1822465581.py:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Step,Training Loss


KeyboardInterrupt: 

In [13]:
# Sauvegarder le modèle et le tokenizer
trainer.save_model("/content/marian-bara-final")
tokenizer.save_pretrained("/content/marian-bara-final")

/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3465: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[54687]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


('/content/marian-bara-final/tokenizer_config.json',
 '/content/marian-bara-final/special_tokens_map.json',
 '/content/marian-bara-final/vocab.json',
 '/content/marian-bara-final/source.spm',
 '/content/marian-bara-final/target.spm',
 '/content/marian-bara-final/added_tokens.json')

In [14]:
!cp -r /content/marian-bara-final /content/drive/MyDrive/marian-bara-final


In [ ]:
#évaluer/tester quelques phrases
# Exemple de phrases test (tu peux en prendre dans ton dataset de test)
phrases_test = [
    ("Salama e", ">>bara<<"),        # malgache officiel -> bara
    ("Nareo koa?", ">>officiel<<"), # bara -> malgache officiel
]

def traduire(texte_source, direction_tag):
    input_text = direction_tag + " " + texte_source
    inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)
    translated = model.generate(**inputs, max_length=128)
    return tokenizer.decode(translated[0], skip_special_tokens=True)

for texte, tag in phrases_test:
    resultat = traduire(texte, tag)
    print(f"Input : {texte} ({tag})")
    print(f"Traduction : {resultat}")
    print("-" * 30)

Input : Salama e (>>bara<<)
Traduction : tena e
------------------------------
Input : Nareo koa? (>>officiel<<)
Traduction : Nareo koa?
------------------------------


In [ ]:
#Recharger le modèle et le tokenizer plus tard
from transformers import MarianMTModel, MarianTokenizer

model_path = "/content/marian-bara-final"  # ou chemin Drive

tokenizer = MarianTokenizer.from_pretrained(model_path)
model = MarianMTModel.from_pretrained(model_path)

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [ ]:
#Utiliser le modèle pour prédire
def traduire(texte_source, direction_tag):
    # Ajouter le tag de langue (ex: >>bara<< ou >>officiel<<)
    input_text = direction_tag + " " + texte_source

    # Tokenisation
    inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)

    # Génération
    translated = model.generate(**inputs, max_length=128)

    # Décodage
    return tokenizer.decode(translated[0], skip_special_tokens=True)

# Exemple
texte_mg = "Inona ny vaovao"
traduction_bara = traduire(texte_mg, ">>bara<<")
print("Traduction :", traduction_bara)

# texte_bara = "aia koa?"
# traduction_mg = traduire(texte_bara, ">>officiel<<")
# print("Traduction inverse :", traduction_mg)

Traduction : Ino ny vaovao


In [ ]:
#Utiliser le modèle pour prédire
def traduire(texte_source, direction_tag):
    # Ajouter le tag de langue (ex: >>bara<< ou >>officiel<<)
    input_text = direction_tag + " " + texte_source

    # Tokenisation
    inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)

    # Génération
    translated = model.generate(**inputs, max_length=128)

    # Décodage
    return tokenizer.decode(translated[0], skip_special_tokens=True)

# Exemple
# texte_mg = "Inona ny vaovao"
# traduction_bara = traduire(texte_mg, ">>bara<<")
# print("Traduction :", traduction_bara)

texte_bara = "mianatsy egny iaho"
traduction_mg = traduire(texte_bara, ">>officiel<<")
print("Traduction inverse :", traduction_mg)

Traduction inverse : mianatra egny aho


In [ ]:
from transformers import MarianMTModel, MarianTokenizer

# Chemin du dossier sauvegardé dans Drive
model_path = "/content/drive/MyDrive/marian-bara-final"

# Charger tokenizer et modèle
tokenizer = MarianTokenizer.from_pretrained(model_path)
model = MarianMTModel.from_pretrained(model_path)
